In [4]:
# Dependencies and Setup
from census import Census
from us import states
from config import (census_key, gkey)
import us
import gmaps
import os
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sodapy import Socrata
import requests
import json

In [5]:
c = Census(census_key, year=2010)
census_data = c.acs5.get(('B01003_001E', 'B17001_002E','B19013_001E'), {'for': 'county:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df= census_df.rename(columns={'B01003_001E': 'Population',
                                      'B17001_002E': 'Poverty Count',
                                      'B19013_001E': 'Median Household Income',
                                      'state':'State',
                                     'county':'County'})
# Add in Poverty Rate (Poverty Count / Population)
census_df['Poverty Rate'] = 100 * \
    census_df['Poverty Count'].astype(
        int) / census_df['Population'].astype(int)

In [6]:
# Clean Census Data
census_df=census_df[census_df.State != '72']
census_df=census_df.reset_index()
state=census_df['State']
county=census_df['County']
popden=[]
baseurl='https://api.census.gov/data/2018/pep/population?get=DENSITY&for=county:'
for i in range(len(state)):
    try:
        url=baseurl+county[i]+'&in=state:'+state[i]+'&key='+census_key
        response = requests.get(url).json()
        response = float(response[1][0])
        popden.append(response)
    except:
        popden.append(np.nan)
    x=c.sf1.state_county('NAME', state[i], county[i])[0]['NAME']
    x=x.split(' County,')
    if len(x)==1:
        x=x[0].split(', ')
    state[i]=x[1]
    county[i]=x[0]
census_df['Population Density']=popden
census_df=census_df[['State','County','Population','Population Density','Median Household Income','Poverty Rate']]
census_df

C:\Users\Henry Randall\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Henry Randall\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,State,County,Population,Population Density,Median Household Income,Poverty Rate
0,California,Merced,250699.0,141.949997,43844.0,21.435267
1,California,Modoc,9605.0,2.241634,34588.0,17.792816
2,California,Mono,13905.0,4.673684,55087.0,11.966918
3,California,Monterey,407435.0,132.733421,59271.0,13.321389
4,California,Napa,134051.0,186.309124,67389.0,9.659010
...,...,...,...,...,...,...
3138,Texas,Gregg,119525.0,452.508511,43367.0,16.066932
3139,Texas,Grimes,26208.0,36.014208,39429.0,13.648504
3140,Texas,Guadalupe,122728.0,230.148941,61274.0,9.550388
3141,Texas,Hale,36041.0,33.672409,36509.0,16.822508


In [7]:
# Read in Medicare.gov hospital compare url: https://data.medicare.gov/resource/xubh-q36u.json
dataset='xubh-q36u'
client = Socrata('data.medicare.gov', None)
hospitals = client.get(dataset,limit=6000)
hospitals_df = pd.DataFrame(hospitals)

# Clean Hospital Data
hospitals_df=hospitals_df[['hospital_name','city','state','county_name','hospital_overall_rating']]
hospitals_df=hospitals_df[hospitals_df.hospital_overall_rating != 'Not Available']
hospitals_df=hospitals_df.reset_index()
hospitals_df=hospitals_df.drop(columns='index')
hospitals_df

,hospital_name,city,state,county_name,hospital_overall_rating
0,GOODLAND REGIONAL MEDICAL CENTER,GOODLAND,KS,SHERMAN,3
1,MARIA PARHAM MEDICAL CENTER,HENDERSON,NC,VANCE,2
2,ALLEGHANY COUNTY MEMORIAL HOSPITAL,SPARTA,NC,ALLEGHANY,2
3,ADVENTIST HEALTHCARE WHITE OAK MEDICAL CENTER,SILVER SPRING,MD,MONTGOMERY,3
4,ADVENTHEALTH LAKE WALES,LAKE WALES,FL,POLK,1
...,...,...,...,...,...
3573,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,DAVIDSON,3
3574,JENNERSVILLE HOSPITAL,WEST GROVE,PA,CHESTER,3
3575,MARINERS HOSPITAL,TAVERNIER,FL,MONROE,5
3576,JEWISH HOSPITAL - SHELBYVILLE,SHELBYVILLE,KY,SHELBY,4


In [8]:
# Define Urls for the Johns Hopkins Data
confirm_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
death_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

# Read in COVID-19 Files
confirm_df=pd.read_csv(confirm_url, error_bad_lines=False)
death_df=pd.read_csv(death_url, error_bad_lines=False)

In [9]:
# Clean Confirm data Rows
confirm_df=confirm_df[confirm_df.Admin2 != 'Unassigned']
confirm_df=confirm_df.dropna()
confirm_df=confirm_df[~confirm_df['Admin2'].astype(str).str.startswith('Out of')]
confirm_df=confirm_df[confirm_df.Admin2 != 'Out of*']
confirm_df=confirm_df.reset_index()

# Clean Death Data Rows
death_df=death_df[death_df.Admin2 != 'Unassigned']
death_df=death_df.dropna()
death_df=death_df[~death_df['Admin2'].astype(str).str.startswith('Out of')]
death_df=death_df[death_df.Admin2 != 'Out of*']
death_df=death_df.reset_index()

In [10]:
death_df

,index,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,...,0,0,0,0,0,0,0,0,0,1
1,6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,...,1,1,1,1,1,1,1,1,1,1
2,7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,...,0,0,0,0,0,0,0,0,0,0
3,8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,...,0,0,0,0,0,0,0,0,0,0
4,9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3142,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,...,0,0,0,0,0,0,0,0,0,0
3138,3143,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,...,0,0,0,0,0,0,0,0,0,0
3139,3144,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,...,0,0,0,0,0,0,0,0,0,0
3140,3145,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,...,0,0,0,0,0,0,0,0,0,0


In [11]:
confirm_df

,index,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,...,6,6,7,8,10,12,12,12,12,12
1,6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,...,15,18,19,20,24,28,29,29,38,42
2,7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,...,0,0,0,0,0,1,2,2,2,3
3,8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,...,0,2,3,3,4,4,4,5,7,8
4,9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,...,5,5,5,5,6,9,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3142,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,...,1,1,2,2,3,3,4,5,5,5
3138,3143,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,...,14,16,20,26,29,32,36,39,40,41
3139,3144,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,...,0,0,0,0,1,1,2,2,3,3
3140,3145,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,...,1,1,1,1,1,2,2,4,4,4
